<a href="https://colab.research.google.com/github/ConstanzaSchibber/capstone_colors/blob/main/notebooks/6_Makeup_App_in_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# App for Makeup Color Search Using Streamlit

In this Jupyter notebook, I development a makeup color search application using Streamlit. This app allows users to filter and find makeup products, offering a more nuanced and extensive color selection process compared to major retailers like Sephora and Ulta.

In the world of makeup, finding the perfect shade can be a challenge. While many online retailers offer basic color filtering, this app takes it a step further by providing a more granular and visually intuitive color selection process.

### Key Features:
1. **Color Filtering**: Users can choose from over 10 color groups, significantly more than typical e-commerce platforms.
2. **Multiple Filter Options**: Products can be filtered by color, brand, and category (lipstick, blush, lipliner, lipgloss).
3. **Visual Color Selection**: Color swatches are provided for users to click and filter by color intuitively.
4. **AI-Powered Color Prediction**: The colors are predicted from product images using advanced methods (detailed in separate notebooks).

### How It Works:
- The app uses Streamlit to create an interactive web interface.
- Color data is pre-processed and grouped by similarity.
- Users interact with color swatches, brand selections, and product categories to filter results.
- The filtered results are displayed in a dynamic, user-friendly table.

### Streamlit

1. **Setup and Styling**:
   - The app uses Streamlit's page configuration to set the layout and sidebar state.
   - Custom CSS is applied to style the page, including the sidebar and table.

2. **Data Loading and Preparation**:
   - The makeup data is loaded from a CSV file and relevant columns are selected.

3. **Dynamic Filters**:
   - The `DynamicFilters` class is used to create filters for category and brand.

4. **Color Swatch Selection**:
   - Color swatches are loaded and displayed as clickable images.
   - When a color is clicked, it filters the dataset by that color.

5. **Result Display**:
   - The filtered results are displayed in a table with product images.

This app leverages Streamlit's interactive features and custom styling to create a user-friendly interface. The color prediction and grouping (done in separate processes) allow for a more refined color selection than typically found in e-commerce platforms.


# Libraries

In [1]:
!pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.1 MB/s eta 0:00:00


In [2]:
!pip install streamlit_dynamic_filters

In [3]:
import streamlit as st
import pandas as pd
from streamlit_dynamic_filters import DynamicFilters

In [4]:
!pip install st_clickable_images

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.0/376.0 kB 6.9 MB/s eta 0:00:00


Read that from public link:

In [8]:
import pandas as pd

url = 'https://drive.google.com/file/d/1DhCe1Kqtckk9zD8FApsswjB_sUrfP_YQ/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)

# Generate Color Swatch

The following will generate the color swatch image and save it in a color folder. It will then be read into the Streamlit app.

In [9]:
from PIL import Image

def generate_color_square(hex_color, size=100, output_path=None):
    # Create an image with the given color
    img = Image.new('RGB', (size, size), hex_color)

    # Save the image
    if output_path:
        img.save(output_path)
    return img

# Example usage
colors = df.median_hex_group.unique()
size = 100  # 100x100 pixels

output_file_list = []

for color in colors:
    color_name = color.lstrip('#')  # Remove the '#' for the filename
    output_file = f"{color_name}.png"
    generate_color_square(color, size, output_file)
    output_file_list.append(output_file)
    print(f"Generated {output_file}")

Generated c13744.png
Generated f3a3a7.png
Generated 8c251e.png
Generated ffd9d5.png
Generated dd4b7a.png
Generated 5e1920.png
Generated da7670.png
Generated f9c2b5.png
Generated e5817c.png
Generated aa1642.png


In [10]:
output_file_list

['c13744.png',
 'f3a3a7.png',
 '8c251e.png',
 'ffd9d5.png',
 'dd4b7a.png',
 '5e1920.png',
 'da7670.png',
 'f9c2b5.png',
 'e5817c.png',
 'aa1642.png']

# Streamlit Code

In [20]:
%%writefile app.py

import streamlit as st
from st_clickable_images import clickable_images
import pandas as pd
from streamlit_dynamic_filters import DynamicFilters
from PIL import Image
import base64


# Set page config
st.set_page_config(
    page_title="Makeup Filter App",
    page_icon="💄",
    layout="wide",
    initial_sidebar_state="expanded",
)


st.markdown(
    """
    <style>

    /* Change background color */
    .main {
        background-color: #f5f5f5; /* Light grey background */
    }

    /* Change font color and style */
    h1, h2, h3, h4, h5, h6, p, li {
        color: #333333; /* Dark grey font color */
        font-family: 'Arial', sans-serif;
    }

    /* Target the entire sidebar */
    [data-testid="stSidebar"] {
        background-color: #210340;
    }

    /* Target all text elements within the sidebar */
    [data-testid="stSidebar"] * {
        color: white !important;
    }

    /* Specific selectors for different text elements */
    [data-testid="stSidebar"] .stMarkdown p {
        color: white !important;
    }

    [data-testid="stSidebar"] .stSelectbox label {
        color: white !important;
    }

    [data-testid="stSidebar"] .stMultiSelect label {
        color: white !important;
    }

    /* Style for sidebar header */
    [data-testid="stSidebarNav"] {
        color: white !important;
    }

    /* Style for dropdown menus in sidebar */
    [data-testid="stSidebar"] .stSelectbox div[data-baseweb="select"] > div,
    [data-testid="stSidebar"] .stMultiSelect div[data-baseweb="select"] > div {
    background-color: #87034e !important;
      }

    /* Style for dropdown options */
    [data-testid="stSidebar"] ul[data-baseweb="menu"] {
    background-color: white !important;
    }

    /* Style for dropdown text */
    [data-testid="stSidebar"] .stSelectbox div[data-baseweb="select"] span,
    [data-testid="stSidebar"] .stMultiSelect div[data-baseweb="select"] span,
    [data-testid="stSidebar"] ul[data-baseweb="menu"] li {
    color: #87034e !important;
    }

    /* Table styles */
    .dataframe {
        color: #333333 !important;
    }
    .dataframe th {
        background-color: #210340;
        color: white !important;
    }
    .dataframe td {
        background-color: #f5f5f5;
    }
    </style>
    """,
    unsafe_allow_html=True
)

with st.sidebar:
    "## Filter Makeup Options!"
# Title and Introduction
st.title("Makeup Filter App 💄")
st.markdown("""
Welcome to the Makeup Filter App! This tool allows you to filter a makeup by color shade, product category, and brand. Use the filters in the sidebar to narrow down the table and explore the data.
""")

# Instructions
st.header("Instructions")
st.markdown("""
1. **Select Filters:** Use the filters in the sidebar to filter the table by color shade, product category, and brand.
2. **View the Table:** The filtered table will be displayed below the filters.
3. **Explore:** You can experiment with different combinations of filters to explore the dataset.

This app is designed to help you easily navigate through the makeup options by providing dynamic filtering options.
""")

## DATA

# Read and sample data

df = pd.read_csv('/content/drive/My Drive/df_for_streamlit.csv')
df = df[df.ground_truth == 1]
df = df[['category', 'brand', 'median_hex_circle', 'Circle', 'img_url']]


# Filter Sidebar

dynamic_filters = DynamicFilters(df=df, filters=['category', 'brand'])
#dynamic_filters.display_filters(location='sidebar')

# save filtered df as new variable
new_df = dynamic_filters.filter_df()

# Color pick

list_images = ['c13744.png',
 'f3a3a7.png',
 '8c251e.png',
 'ffd9d5.png',
 'dd4b7a.png',
 '5e1920.png',
 'da7670.png',
 'f9c2b5.png',
 'e5817c.png',
 'aa1642.png']

images = []
for file in list_images:
    with open(file, "rb") as image:
        encoded = base64.b64encode(image.read()).decode()
        images.append(f"data:image/jpeg;base64,{encoded}")

### Create color bar
clicked = clickable_images(
    images,
    titles=[f"Image #{str(i)}" for i in range(10)],
    div_style={"display": "flex", "justify-content": "center", "flex-wrap": "wrap"},
    img_style={"margin": "5px", "height": "50px"},
)

## Filter by color
if clicked == 0:
  cat = df.median_hex_circle.unique()[0]
  new_df = new_df[new_df.median_hex_circle == cat]
  dynamic_filters = DynamicFilters(df=new_df, filters=['category', 'brand'])

elif clicked == 1:
  cat = df.median_hex_circle.unique()[1]
  new_df = new_df[new_df.median_hex_circle == cat]
  dynamic_filters = DynamicFilters(df=new_df, filters=['category', 'brand'])

elif clicked == 2:
  cat = df.median_hex_circle.unique()[2]
  new_df = new_df[new_df.median_hex_circle == cat]
  #dynamic_filters = DynamicFilters(df=new_df, filters=['category', 'brand'])

elif clicked == 3:
  cat = df.median_hex_circle.unique()[3]
  new_df = new_df[new_df.median_hex_circle == cat]
  #dynamic_filters = DynamicFilters(df=new_df, filters=['category', 'brand'])

elif clicked == 4:
  cat = df.median_hex_circle.unique()[4]
  new_df = new_df[new_df.median_hex_circle == cat]
  #dynamic_filters = DynamicFilters(df=new_df, filters=['category', 'brand'])

elif clicked == 5:
  cat = df.median_hex_circle.unique()[5]
  new_df = new_df[new_df.median_hex_circle == cat]
  #dynamic_filters = DynamicFilters(df=new_df, filters=['category', 'brand'])

elif clicked == 6:
  cat = df.median_hex_circle.unique()[6]
  new_df = new_df[new_df.median_hex_circle == cat]
  #dynamic_filters = DynamicFilters(df=new_df, filters=['category', 'brand'])

elif clicked == 7:
  cat = df.median_hex_circle.unique()[7]
  new_df = new_df[new_df.median_hex_circle == cat]
  #dynamic_filters = DynamicFilters(df=new_df, filters=['category', 'brand'])

elif clicked == 8:
  cat = df.median_hex_circle.unique()[8]
  new_df = new_df[new_df.median_hex_circle == cat]
  #dynamic_filters = DynamicFilters(df=new_df, filters=['category', 'brand'])

elif clicked == 9:
  cat = df.median_hex_circle.unique()[9]
  new_df = new_df[new_df.median_hex_circle == cat]
  #dynamic_filters = DynamicFilters(df=new_df, filters=['category', 'brand'])

dynamic_filters.display_filters(location='sidebar')

# Converting links to html tags
def path_to_image_html(path):
    return '<img src="' + path + '" height="60" >'


if len(new_df) < len(df):
    # Apply the color_square function to the 'District' column
    #new_df['District'] = new_df['District'].apply(color_square)
    new_df = new_df.sort_values(by = ['category','brand'])
    # Display the dataframe with HTML
    #st.write(new_df.to_html(escape=False, index=False), formatters=dict(img_url=path_to_image_html), unsafe_allow_html=True)

    def convert_df(input_df):
     # IMPORTANT: Cache the conversion to prevent computation on every rerun
     return input_df.to_html(escape=False, formatters=dict(img_url=path_to_image_html))

    html = convert_df(new_df)

    st.markdown(
    html,
    unsafe_allow_html=True
    )

else:
    st.write("Please select at least one filter to display the data.")




Overwriting app.py


# Deploying and Sharing the Streamlit App

The following set of commands is typically used to run a Streamlit app and make it accessible over the internet. Here's a brief explanation:

- `!npm install localtunnel`: Installs the localtunnel package, which is used to expose local servers to the internet.


- `!curl ipv4.icanhazip.com`: Retrieves the public IP address of the machine running the command.


- `!streamlit run app.py &>./logs.txt &`: Runs the Streamlit app defined in `'app.py'.`

- `&>./logs.txt`: redirects both standard output and error to a log file.
The & at the end runs the command in the background.


- `npx localtunnel --port 8501`: Uses localtunnel to expose the Streamlit app (which typically runs on port 8501) to the internet.


This combination of commands is often used in Google Colab to run and share Streamlit apps. It allows others to access your locally running Streamlit app through a public URL provided by localtunnel.

In [12]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [13]:
# Your public ip is the password to the localtunnel
!curl ipv4.icanhazip.com

35.197.97.31


After running this, you will have a link. Click the link and input the IP address provided right above into the text box, click enter.

In [21]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

your url is: https://eighty-memes-pump.loca.lt
^C
